In [1]:
!pip install -U duckduckgo-search
!pip install langchain_community
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [61]:
import requests
import json
import os
import pandas as pd
import http.client
import re
import time
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

## Set LLM API credentials

In [ ]:
# Set Azure OpenAI credentials
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_KEY"] = "" #key here
os.environ["OPENAI_API_BASE"] = "https://Phi-3-5-mini-instruct-iplvi.eastus2.models.ai.azure.com/chat/completions"
os.environ["OPENAI_API_VERSION"] = "2024-02-15-preview"  # Update this to your Azure API version
os.environ["X-API-KEY"] = '' # your https://serper.dev key

## Get data of fortune 500 companies

In [4]:
url = "https://fortune.com/ranking/global500/"
tables = pd.read_html(url)
df_f5_2024 = tables[0]

In [5]:
url = "https://fortune.com/ranking/global500/2004/"
tables = pd.read_html(url)
df_f5_2004 = tables[0]

## Write functions to help

In [6]:
search = DuckDuckGoSearchRun()

In [20]:
def g_search(query):

  conn = http.client.HTTPSConnection("google.serper.dev")
  X_API_KEY = os.environ["X-API-KEY"]

  payload = json.dumps({
    "q": f"""{query}"""
  })

  headers = {
    'X-API-KEY': X_API_KEY,
    'Content-Type': 'application/json'
  }

  conn.request("POST", "/search", payload, headers)
  res = conn.getresponse()
  data = res.read()
  search_results = data.decode("utf-8")
  return search_results

In [21]:
def search_ceo(company):
  #ceo_search = search.run(f"Who is the CEO of {company}?")
  ceo_search = g_search(f"Who is the CEO of {company}?")
  return ceo_search

In [22]:
def search_ceo_background(company, CEO):
  #ceo_background = search.run(f"{CEO}, CEO of {company} eduction background")
  ceo_background = g_search(f"{CEO}, CEO of {company} eduction and background")
  return ceo_background

In [23]:
def search_ceo_startdate(company, CEO):
  #ceo_startdate = search.run(f"when was {CEO} appointed as CEO of {company} ")
  ceo_startdate = g_search(f"when was {CEO} appointed as CEO of {company} ")
  return ceo_startdate

In [24]:
#does Mirko Bibic, current CEO of Bell Canada Have a Legal background?
def search_ceo_legal_background(company, CEO, profession='Legal'):
  Search_query = f"does {CEO}, CEO of {company} have a {profession} background"
  ceo_background = g_search(Search_query)
  return ceo_background

In [25]:
def ask_phi(input_query):

    # Define the API endpoint and headers
    api_endpoint = os.getenv("OPENAI_API_BASE")
    api_key = os.getenv("OPENAI_API_KEY")  # Ensure your Azure API key is set in the environment

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}",
        "api-version": "2023-05-15"
    }

    # Define the request payload
    payload = {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"""{input_query}"""}
        ],
        "max_tokens": 100,
        "temperature": 0.3,
        "top_p": 0.95
    }

    # Make the POST request
    response = requests.post(api_endpoint, json=payload, headers=headers)

    # Handle the response
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        return (f"Error: {response.status_code}, {response.text}")

In [53]:
def extract_json(response, feild):
    content = response['choices'][0]['message']['content']
    content = str(content)
    # Extract JSON from the response content
    try:
      json_match = re.search(r'{.*?}', content, re.DOTALL)
      if json_match:
        json_str = json_match.group(0)
        # Parse the JSON string
        data = json.loads(json_str)
        return data[feild]
      else:
        return json.loads(content)[feild]
    except:
      return "No details found"

In [67]:
def process_ceo_information(company_name, profession='Legal'):
    # Step 1: Search for the CEO
    ceo_name_search = search_ceo(company_name)

    # Step 2: Create a prompt to identify the CEO's full name
    ceo_name_prompt = f"""{ceo_name_search} "Identify the CEO of {company_name} from the above text and return only the CEO's full name in a JSON format. Key = 'CEO_Name'"""
    ceo_name_response = ask_phi(ceo_name_prompt)
    ceo_name = extract_json(ceo_name_response, 'CEO_Name')

    # Step 3: Search for the CEO's background
    ceo_background_info = search_ceo_background(ceo_name, company_name)

    # Step 4: Create a prompt to classify the CEO's background
    ceo_background_prompt = f"""{ceo_background_info} /// Classify the CEO background as one of the following: Technology, Medicine, Accounting, Finance, Strategy Consulting, Law, Engineering, Business Adminstration,  Other. Return only the background classification in a JSON format. Key = 'CEO_Background'"""
    ceo_background_response = ask_phi(ceo_background_prompt)
    ceo_background = extract_json(ceo_background_response, 'CEO_Background')

    # Step 5: Search for the start year of the CEO's appointment
    ceo_start_year_info = search_ceo_startdate(company_name, ceo_name)

    # Step 6: Create a prompt to identify the CEO's start year
    ceo_start_year_prompt = f"""{ceo_start_year_info} /// Identify just the year {ceo_name} was appointed as CEO of the company {company_name}. Return only the year as a number in a JSON format. Key = 'Appointed_Year'"""
    ceo_start_year_response = ask_phi(ceo_start_year_prompt)
    appointed_year = extract_json(ceo_start_year_response, 'Appointed_Year')

    # Step 7: Search if CEO has {legal} backgdound
    ceo_legal_background_info = search_ceo_legal_background(company_name, ceo_name, profession )

    # Step 8: Create a prompt to identify the CEO's {Legal} Background
    ceo_legal_background_prompt = f"""{ceo_legal_background_info} ///does {ceo_name}, current CEO of {company_name} have a {profession} background? ONLY RETURN Yes or No in a JSON format. Key = '{profession}_Background'"""
    ceo_legal_background_response = ask_phi(ceo_legal_background_prompt)
    profession_background = extract_json(ceo_legal_background_response, f'{profession}_Background')

    # Step 9: Create a prompt to identify the CEO's {consluting} Background (an after thought does not fit in to the pattern# code can be improved)
    ceo_consluting_background_prompt = f"""{ceo_background_info} ///has {ceo_name}, current CEO of {company_name} worked for consulting companies? ONLY RETURN Yes or No in a JSON format. Key = 'Consulting_Background'"""
    ceo_consluting_background_response = ask_phi(ceo_consluting_background_prompt)
    consulting_background = extract_json(ceo_consluting_background_response, 'Consulting_Background')

    return {
        'CEO Name': ceo_name,
        'CEO Background': ceo_background,
        'Appointed Year': appointed_year,
        f'{profession} Background': profession_background,
        'Consulting Background': consulting_background,
        'Search Results': {
            'CEO Name Search': ceo_name_search,
            'CEO Background Info': ceo_background_info,
            'CEO Start Year Info': ceo_start_year_info,
            'CEO Legal Background Info': ceo_legal_background_info
        },
        'Ask Phi Responses': {
            'CEO Name Response': ceo_name_response,
            'CEO Background Response': ceo_background_response,
            'CEO Start Year Response': ceo_start_year_response,
            f'{profession} Background Response': ceo_legal_background_response,
            'Consulting Background Response': ceo_consluting_background_response
        }
    }

# Example usage
# company_info = process_ceo_information('Walmart')
# print(company_info)

In [60]:
company_info = process_ceo_information('Tata')
print(company_info)

{'CEO Name': 'Natarajan Chandrasekaran', 'CEO Background': 'Technology', 'Appointed Year': 2009, 'Legal Background': 'No', 'Consulting Background': 'Yes', 'Search Results': {'CEO Name Search': '{"searchParameters":{"q":"Who is the CEO of Tata?","type":"search","engine":"google"},"knowledgeGraph":{"title":"Natarajan Chandrasekaran","type":"Chairperson of the Tata Group","imageUrl":"https://encrypted-tbn0.gstatic.com/licensed-image?q=tbn:ANd9GcRmpbRDSH9s2OfUa6q4BpFHFfcl-gtBncTICs969Fe1g4oXFBEZefDRzrwf70dYsMwt6CCH&s=19","description":"Natarajan Chandrasekaran is an Indian businessman, and the chairman of Tata Sons and Tata Group. He was chief operating officer and executive director of Tata Consultancy Services, where in 2009, he became chief executive officer. He was also the...","descriptionSource":"Wikipedia","descriptionLink":"https://en.wikipedia.org/wiki/Natarajan_Chandrasekaran","attributes":{"Born":"1963 (age 61\xa0years), Mohanur, India","Awards":"Padma Bhushan and CNN-IBN Indian

In [ ]:
# Saving data to drive just to be safe
csv_file_path = '/content/drive/MyDrive/Blog/Fortune 500 Lawyers/fortune_500_data.csv'  # Update with your desired file paths

for index, row in df_f5_2024.iterrows():
  company = row['RemoveName']
  try:
    ceo_information = process_ceo_information(company, 'Legal')
    df_f5_2024.loc[index, 'CEO Name'] = ceo_information['CEO Name']
    df_f5_2024.loc[index, 'CEO Background'] = ceo_information['CEO Background']
    df_f5_2024.loc[index, 'Appointed Year'] = ceo_information['Appointed Year']
    df_f5_2024.loc[index, 'Legal Background'] = ceo_information['Legal Background']
    df_f5_2024.loc[index, 'Consulting Background'] = ceo_information['Consulting Background']
    #time.sleep(60)

    # Saving restuls to drive
    txt_file_path = f'/content/drive/MyDrive/Blog/Fortune 500 Lawyers/{company}_ceo_information.txt'
    with open(txt_file_path, 'w') as f:
      f.write(str(company_info))
    if index % 10 == 0:
      df_f5_2024.to_csv(csv_file_path, index=False)

  except Exception as e:
    print(f"Error processing information for {company}: {e}")

In [65]:
df_f5_2024.to_csv(csv_file_path, index=False)